In [24]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib

## From stack exchange
https://stackoverflow.com/questions/1936466/beautifulsoup-grab-visible-webpage-text

In [31]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u"\n".join(t.strip() for t in visible_texts)


In [38]:
page = urllib.request.urlopen('https://www.facebook.com/legal/FB_Work_Privacy')
html = page.read()
page.code

200

In [73]:
#Parse out the URL
url = 'http://med.net/this/that?thisthat/something.html'
'.'.join(url.split('//')[-1].split('/')[0].split('.')[-2:])
# >>> url.split("//")[-1].split("/")[0]
# 'stackoverflow.com'
# >>> url = "stackoverflow.com/questions/9626535/get-domain-name-from-url"
# >>> url.split("//")[-1].split("/")[0]
# 'stackoverflow.com'

'med.net'

## Define a new url segmenter to do all this.

In [39]:
import urllib
from bs4 import BeautifulSoup


def url_input_parser(url):
    """
    Input is url as a string. Parses body text and returns
    string of all paragraph elements in doc.
    Secondarily, searches the input url for its domain and returns
    domain in the form 'google.com'.
    """
    
    # Request URL
    page = urllib.request.urlopen(url)

    # Check we got a good response, otherwise complain
    if str(page.code)[0] != '2':
        e = urllib.error.URLError('The server returned status code ' + str(page.code))
        raise e
    
    content = page.read()
    
    # Parse URL paragraphs
    pars = text_from_html(content)

    # Parse domain
    domain = '.'.join(url.split('//')[-1].split('/')[0].split('.')[-2:])
    return pars, domain


In [44]:
testurl = 'https://www.facebook.com/legal/FB_Work_Privacy'
text, domain = url_input_parser(testurl)
print(text)


Jump to
Sections of this page
Accessibility Help
Press
alt
+
/
to open this menu
Facebook
Join or Log Into Facebook
Email or Phone
Password
Forgot account?
Log In
Do you want to join Facebook?
Sign Up
Sign Up
Workplace Privacy Policy
Effective Date: September 30, 2016
Workplace allows users to more effectively collaborate and share information at work. This Privacy Policy describes how and when your information is collected, used, and shared by Facebook when you, your colleagues, or other users use the Workplace platform (which includes the Workplace websites, apps and related online services that link to this Privacy Policy, which we call the “Services”).
The Services are intended for use by organizations and in accordance with their instructions. If you are using the Services in a workplace or on a device or account issued to you by your employer or another organization (your “Organization”), that Organization likely has its own policies regarding storage, access, modification, dele

## Parsing the results

In [45]:
def text_paragraph_segmenter(doc):
    """
    input: doc as string
    output: list of paragraphs with blank lines removed
    """

    lines = doc.split('\n')
    segs = list()
    segs.append(lines[0])
    c = 0
    for i in range(1, len(lines)):
        if len(lines[i]) < 75:  # Less than 75 chars wide in line join to prev line
            segs[c] = ' '.join([segs[c], lines[i]])
        else:
            segs.append(lines[i])
            c += 1

    return [line for line in segs if line.strip() != '']


In [46]:
import pandas as pd
segments = text_paragraph_segmenter(text)

In [48]:
for s in segments:
    print(s)

 Jump to Sections of this page Accessibility Help Press alt + / to open this menu Facebook Join or Log Into Facebook Email or Phone Password Forgot account? Log In Do you want to join Facebook? Sign Up Sign Up Workplace Privacy Policy Effective Date: September 30, 2016
Workplace allows users to more effectively collaborate and share information at work. This Privacy Policy describes how and when your information is collected, used, and shared by Facebook when you, your colleagues, or other users use the Workplace platform (which includes the Workplace websites, apps and related online services that link to this Privacy Policy, which we call the “Services”).
The Services are intended for use by organizations and in accordance with their instructions. If you are using the Services in a workplace or on a device or account issued to you by your employer or another organization (your “Organization”), that Organization likely has its own policies regarding storage, access, modification, dele

## Pre-processing policy before the good stuff

In [124]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string
def text_process_policy(doc):
    """
    Takes in doc as string and returns a processed string
    by performing the following steps:
    1. remove HTML tags
    2. remove punctuation
    3. remove stopwords
    4. stemming
    5. remove any blank lines
    """
    sn = SnowballStemmer(language='english')

    s = re.sub(r'\<.*\>','',doc)
    lst = [char for char in s if char not in string.punctuation]
    lst = ''.join(lst)
    lst = [word.lower() for word in lst.split() if word.lower() not in stopwords.words('english')]
    lst = [sn.stem(word) for word in lst]
    return ' '.join(lst)


In [125]:
segments_processed = [text_process_policy(segment) for segment in segments]
print(segments_processed)

['', 'pleas enabl javascript browser upgrad javascriptcap browser regist facebook', '', '', 'click creat account agre', '', 'term', '', 'read', '', 'data polici', '', 'includ', '', 'cooki use', '', 'may receiv sms notif facebook opt time', '']


In [133]:
no_blanks = [segment for segment in segments_processed if segment.strip() != '']
no_blanks

['pleas enabl javascript browser upgrad javascriptcap browser regist facebook',
 'click creat account agre',
 'term',
 'read',
 'data polici',
 'includ',
 'cooki use',
 'may receiv sms notif facebook opt time']

In [134]:
no_shorties = [segment for segment in no_blanks if len(segment) >= 20]
no_shorties

['pleas enabl javascript browser upgrad javascriptcap browser regist facebook',
 'click creat account agre',
 'may receiv sms notif facebook opt time']